In [10]:
import numpy as np
from numba import njit
from functools import partial

import pandas as pd
import vectorbt as vbt
import vectorbt as vbt
from vectorbt.utils.colors import adjust_opacity
from vectorbt.utils.enum_ import map_enum_fields
from vectorbt.base.reshape_fns import broadcast, flex_select_auto_nb, to_2d_array
from vectorbt.portfolio.enums import SizeType, Direction, NoOrder, OrderStatus, OrderSide
from vectorbt.portfolio import nb

import talib 

import sys
import os
function_essential_path = r"H:\Desktop\Environement_Trading_Developement\FunctionEssential"
function_s_score_path = r'H:\Desktop\Environement_Trading_Developement\Indicateur'
sys.path.append(function_essential_path)
sys.path.append(function_s_score_path)
import utils as us
import s_score as score
import VPN as vpn

In [2]:
##FONCTION##

In [38]:
def entries_trades(s_score, VPN, close, seuil_sscore, seuil_vpn):
    entries_long = [False] * len(close)
    entries_short = [False] * len(close)
    couldown = 0  

    for i in range(len(close)):

        if (s_score[i] >= seuil_sscore) & (VPN[i] < seuil_vpn) & (couldown >= 10):
            entries_short[i] = True
            couldown = 0

        if (s_score[i] <= -1*seuil_sscore) & (VPN[i] < seuil_vpn) & (couldown >= 10):
            entries_long[i] = True
            couldown = 0

        couldown += 1  

    return entries_long, entries_short


In [4]:
##INDICATEUR FACTORY##

In [40]:
##CLASS STRATEGIE##
@njit
def pre_sim_func_nb(c):
    entry_price = np.full(c.target_shape[1], np.nan, dtype=np.float64)  
    return (entry_price,)



@njit
def order_func_nb(c, entry_price, long, short, stop_loss, take_profit, size):
    price_now = nb.get_elem_nb(c, c.close)
    entry_long_now = nb.get_elem_nb(c, long)
    entry_short_now = nb.get_elem_nb(c, short)

    # Vérifie si nous devons entrer en position longue
    if entry_long_now:
        if c.position_now == 0:  # Aucune position ouverte
            return nb.order_nb(
                size,
                price=price_now,
                direction=Direction.LongOnly,
                fees=0.0005
                )
        elif c.position_now < 0:  # Position courte ouverte, on doit d'abord la fermer
            return nb.order_nb(
                size=-size,  # On ferme la position courte
                price=price_now,
                direction=Direction.ShortOnly,
                fees=0.0005
                )  # Puis, on ouvre une position longue

    # Vérifie si nous devons entrer en position courte
    if entry_short_now:
        if c.position_now == 0:  # Aucune position ouverte
            return nb.order_nb(
                size,
                price=price_now,
                direction=Direction.ShortOnly,
                fees=0.0005
                )
        elif c.position_now > 0:  # Position longue ouverte, on doit d'abord la fermer
            return nb.order_nb(
                size=-size,  # On ferme la position longue
                price=price_now,
                direction=Direction.LongOnly,
                fees=0.0005
                )  # Puis, on ouvre une position courte

    # Gestion des positions longues (on ferme la position si on atteint le stop-loss ou le take-profit)
    if c.position_now > 0:
        SL = entry_price[c.col] * (1 - stop_loss)
        TP = entry_price[c.col] * (1 + take_profit)
        
        if price_now >= TP:  # Si le prix atteint le TP, on ferme la position
            return nb.order_nb(
                -size,
                price=TP,
                direction=Direction.LongOnly,
                fees=0.0005
                )
        elif price_now <= SL:  # Si le prix atteint le SL, on ferme la position
            return nb.order_nb(
                -size,
                price=SL,
                direction=Direction.LongOnly,
                fees=0.0005
                )
    
    # Gestion des positions courtes (on ferme la position si on atteint le stop-loss ou le take-profit)
    if c.position_now < 0:
        SL = entry_price[c.col] * (1 + stop_loss)
        TP = entry_price[c.col] * (1 - take_profit)
        
        if price_now >= SL:  # Si le prix atteint le SL, on ferme la position
            return nb.order_nb(
                -size,
                price=SL,
                direction=Direction.ShortOnly)
        elif price_now <= TP:  # Si le prix atteint le TP, on ferme la position
            return nb.order_nb(
                -size,
                price=TP,
                direction=Direction.ShortOnly)

    return NoOrder

@njit
def post_order_func_nb(c, entry_price):
    if c.order_result.status == OrderStatus.Filled:
        entry_price[c.col] = c.order_result.price



class Strategie():


    def __init__(self,data,tickers,frequence_exec,frequence,multitimeframe):
        self.data = data
        self.tickers = tickers
        self.frequence_exec = frequence_exec
        self.frequences = frequence
        self.multitimeframe = multitimeframe   ## Si 0 pas de multitimeframe si 1 multitimeframe

    def backtest(self,s_score):
        entries_long_df = pd.DataFrame()
        entries_short_df = pd.DataFrame()
        exits_long_df = pd.DataFrame()
        exits_short_df = pd.DataFrame()
        close_df = pd.DataFrame()

        for ticker in self.tickers:
            data = self.data[ticker]
            s_score_ajuster = s_score[1]
            s_score_ajuster = pd.Series(s_score_ajuster)
            VPN = vpn.calculate_vpn(np.array(data['high']),np.array(data['low']),np.array(data['close']),np.array(data['volume']),30)
            long,short = entries_trades(s_score_ajuster,VPN,data['close'],3,10)

            long = pd.Series(long, index=data['close'].index)  # Assurer que l'index est celui de close
            short = pd.Series(short, index=data['close'].index)  # Assurer que l'index est celui de close

            # Réinitialiser les index
            long = long.reset_index(drop=True)
            short = short.reset_index(drop=True)
            close = data['close'].reset_index(drop=True)

            SL = 0.02
            TP = 0.04
            size = 1

            #Reset les index pour tout normaliser
            long = long.reset_index(drop=True)
            short = short.reset_index(drop=True)

            # exits_long = exits_long.reset_index(drop=True)
            # exits_short = exits_short.reset_index(drop=True)
            # close = data['close'].reset_index(drop=True)
            # #enregistrer dans des dataframes 
            # entries_long_df[ticker] = long
            # entries_short_df[ticker] = short
            # exits_long_df[ticker] = exits_long
            # exits_short_df[ticker] = exits_short
            # close_df[ticker] = close
            print(f'---TICKER : {ticker}-----')
        portfolio = vbt.Portfolio.from_order_func(
            close,
            order_func_nb,  # Fonction qui gère les ordres
            long.values,  # Signaux longs
            short.values,  # Signaux courts
            SL,  # Stop loss
            TP,  # Take profit
            size,  # Taille des positions
            freq=self.frequence_exec,  # Fréquence d'exécution
            pre_sim_func_nb=pre_sim_func_nb,  # Fonction appelée avant la simulation
            post_order_func_nb=post_order_func_nb,  # Fonction appelée après chaque ordre
            broadcast_named_args=dict(  # Diffusion des arguments pour les fonctions
                long=long,
                short=short,
                stop_loss=SL,
                take_profit=TP,
                size=size
            )
        )

        us.get_pnl(portfolio)
        us.generate_portfolio_report_order_function(portfolio, close)
        return portfolio



    def set_data(self,data):
        self.data = data
    
    def set_ticker(self,ticker):
        self.tickers = ticker

In [15]:
##UTILISATION DE LA STRAT##
s_score_value_all = {}
data_all = {}
tickers = ['FDAX','YM','NK','SP']
for ticker in tickers:
    data = us.get_data_indice([ticker],'H4',"2000-01-01","2016-01-01")
    data_whithout_index = data[ticker].reset_index()
    data_all[ticker]= data
    s_score_value = score.mean_reversion(data[ticker]['close'], data_whithout_index['Datetime'],85,"VOL", 0.5,"EGARCH")
    s_score_value_all[ticker] = s_score_value




Estimation des paramètres OU:   0%|          | 0/15770 [00:00<?, ?it/s]

  0%|          | 0/15770 [00:00<?, ?it/s]

Estimation des paramètres OU:   0%|          | 0/10918 [00:17<?, ?it/s]
H:\Desktop\Environement_Trading_Developement\Indicateur\s_score.py:31: RuntimeWarning: invalid value encountered in sqrt
  sigma_eq = sigma / np.sqrt(2 * theta)
Calcul de la volatilité conditionnelle: 100%|██████████| 15855/15855 [00:00<00:00, 44321.32it/s]


15854


  0%|          | 0/10917 [00:00<?, ?it/s]

...

Estimation des paramètres OU:   0%|          | 0/10917 [00:06<?, ?it/s]
H:\Desktop\Environement_Trading_Developement\Indicateur\s_score.py:31: RuntimeWarning: invalid value encountered in sqrt
  sigma_eq = sigma / np.sqrt(2 * theta)
c:\Users\CYTech Student\AppData\Local\Programs\Python\Python311\Lib\site-packages\arch\univariate\base.py:768: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
Calcul de la volatilité conditionnelle: 100%|██████████| 10335/10335 [00:00<00:00, 70371.26it/s]


10334


  0%|          | 0/10918 [00:00<?, ?it/s]

Estimation des paramètres OU:   0%|          | 0/10250 [00:05<?, ?it/s]
H:\Desktop\Environement_Trading_Developement\Indicateur\s_score.py:31: RuntimeWarning: invalid value encountered in sqrt
  sigma_eq = sigma / np.sqrt(2 * theta)
Calcul de la volatilité conditionnelle:   0%|          | 0/11003 [00:00<?, ?it/s]c:\Users\CYTech Student\AppData\Local\Programs\Python\Python311\Lib\site-packages\arch\univariate\base.py:768: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
Calcul de la volatilité conditionnelle: 100%|██████████| 11003/11003 [00:00<00:00, 220707.45it/s]

11002


In [41]:

strat = Strategie(None,None,"240m","240m",0)
for ticker in tickers:

    strat.set_data(data_all[ticker])
    strat.set_ticker([ticker])

    strat.backtest(s_score_value_all[ticker])

---TICKER : FDAX-----



-----------------------REPORT METRICS-----------------------


Total Return : -178.64%
Benchmark return : 42.31%
CAGR : nan%
Volatility : 2893.33%
Sharpe Ratio : 0.41
Benchmark sharpe ratio : 0.13
Max Drawdown: -339.46%
Calmar Ratio : nan
Beta : -0.47


--------------- REPORT METRICS TRADES--------------------


Total trades : 15
Total long trades 13
Total short trades 2
Win rate : 0.2857142857142857%
Wining streak : 2.0
Loosing streak : 4.0
Average winning trade : 4.0%
Average losing trade : -7.000000000000001%
Profit factor : 0.08
Expectancy : -11.91%

-----------------------------------------------------------

---TICKER : YM-----



-----------------------REPORT METRICS-----------------------


Total Return : 565.32%
Benchmark return : 126.53%
CAGR : 189.99%
Volatility : 525.31%
Sharpe Ratio : 0.81
Benchmark sharpe ratio : 0.39
Max Drawdown: -944.02%
Calmar Ratio : 0.2
Beta : -1.81


--------------- REPORT METRICS TRADES--------------------


Total trades : 25
Total long trades 22
Total short trades 3
Win rate : 0.375%
Wining streak : 2.0
Loosing streak : 7.0
Average winning trade : 4.0%
Average losing trade : -2.0%
Profit factor : 2.26
Expectancy : 22.61%

-----------------------------------------------------------

---TICKER : NK-----



-----------------------REPORT METRICS-----------------------


Total Return : 0.0%
Benchmark return : 111.32%
CAGR : 0.0%
Volatility : 0.0%
Sharpe Ratio : inf
Benchmark sharpe ratio : 0.29
Max Drawdown: 0.0%
Calmar Ratio : nan
Beta : 0.0


--------------- REPORT METRICS TRADES--------------------


Total trades : 0
Total long trades 0
Total short trades 0
Win rate : nan%
Wining streak : nan
Loosing streak : nan
Average winning trade : nan%
Average losing trade : nan%
Profit factor : nan
Expectancy : nan%

-----------------------------------------------------------

---TICKER : SP-----



-----------------------REPORT METRICS-----------------------


Total Return : -120.1%
Benchmark return : 155.69%
CAGR : nan%
Volatility : 446.68%
Sharpe Ratio : -0.35
Benchmark sharpe ratio : 0.41
Max Drawdown: -104.25%
Calmar Ratio : nan
Beta : -1.24


--------------- REPORT METRICS TRADES--------------------


Total trades : 16
Total long trades 11
Total short trades 5
Win rate : 0.4%
Wining streak : 2.0
Loosing streak : 4.0
Average winning trade : 4.0%
Average losing trade : -4.0%
Profit factor : 0.16
Expectancy : -7.51%

-----------------------------------------------------------

